In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import os, fnmatch
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures 
import seaborn as sns
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
%matplotlib inline

In [2]:
all_spec_files = []
listOfFiles = os.listdir('./spec')
pattern = "*.spec"
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
            all_spec_files.append(entry)

all_fits_files = []
listOfFiles = os.listdir('./plate')
pattern = "*.fits"
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
            all_fits_files.append(entry)
            
            
all_spec_files.sort()
all_fits_files.sort()

In [31]:
for plate in range(len(all_fits_files)):
    plate_id = all_fits_files[plate].split('_')[0]
    print(plate_id)
    current_fits_file= WCS("plate/"+all_fits_files[plate])
    current_plate_objects = []
    
    current_spec_list = []
    first_check = False
    for i in range(len(all_spec_files)):
        spec_plate_id = all_spec_files[i].split('-')[0]
        if spec_plate_id == plate_id:
            current_spec_list.append(all_spec_files[i])
            first_check = True
        elif first_check:
            break
    
    for i in range(len(current_spec_list)):
        obj_vector = []
        index = 0
        arr_x = []
        arr_y = []
        f = open("spec/"+current_spec_list[i],'r')
        for line in f:
            if line[0] != "#":
                arr_y.append(float(line.split(" ")[2]))
                arr_x.append(float(line.split(" ")[4].split("\n")[0]))
            elif (line[0] == "#" and line[1] != "#"):
                if(line[0:3:] != "# r"):
                    str_arr = line.split(" ")
    #               print(str_arr)
                    if(index <= 6):
                        if index == 4 or index == 5 :
                            obj_vector.append(float(str_arr[-1].split("\n")[0]))
                        else:
                            obj_vector.append(str_arr[-1].split("\n")[0])
                    else:
                        value = int(float(str_arr[-2]))
                        obj_vector.append(value)

                    index += 1

        obj_ra = obj_vector[2]
        obj_dec = obj_vector[3]
        
        c = SkyCoord(obj_ra, obj_dec, unit=(u.hourangle, u.deg))
        dx, dy = current_fits_file.all_world2pix(c.ra, c.dec, 1)
        #print(dx, dy)
        obj_vector.append(int(dx))
        obj_vector.append(int(dy))
        current_plate_objects.append(obj_vector)
        #print(obj_vector)
        #print(ra, dec)
    #print(current_plate_objects)
    current_df = pd.DataFrame(current_plate_objects)
    current_df.rename({0: "plate",
                 1: "object_id",
                 2: "ra",
                 3: "dec",
                 4: "mag_B",
                 5: "mag_R",
                 6: "spectrum_class",
                 7: "spectrum_length",
                 8: "red_bord_position",
                 9: "dx",
                 10: "dy"}, inplace=True, axis=1)
    print(current_df)
    current_df.to_csv(plate_id+"_dataset.csv")
    

fbs1309
      plate                object_id            ra           dec  mag_B  \
0   fbs1309  DFBSJ041556.67-005308.4  04:15:56.676  -00:53:08.43  17.36   
1   fbs1309  DFBSJ041557.48-004852.5  04:15:57.480  -00:48:52.52  16.06   
2   fbs1309  DFBSJ041603.47-004852.8  04:16:03.471  -00:48:52.83  16.78   
3   fbs1309  DFBSJ041608.03-004746.6  04:16:08.030  -00:47:46.62  15.80   
4   fbs1309  DFBSJ041614.00-005245.9  04:16:14.003  -00:52:45.99  14.91   
5   fbs1309  DFBSJ041614.19-005607.3  04:16:14.193  -00:56:07.33  14.28   
6   fbs1309  DFBSJ041616.68-005512.1  04:16:16.683  -00:55:12.12  15.10   
7   fbs1309  DFBSJ041618.47-005139.8  04:16:18.479  -00:51:39.87  14.62   
8   fbs1309  DFBSJ041619.14-005522.1  04:16:19.140  -00:55:22.19  11.87   
9   fbs1309  DFBSJ041619.80-005501.2  04:16:19.809  -00:55:01.25  15.50   
10  fbs1309  DFBSJ041621.93-004800.0  04:16:21.936  -00:48:00.04  16.52   

    mag_R spectrum_class  spectrum_length  red_bord_position    dx    dy  
0   16.15       